In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import matplotlib.pyplot as plt

RUTA_EXCEL = r".\DatosAnypsa.xlsx"
RUTA_SALIDA = r".\DatosAnypsa_Resultados.xlsx"
RUTA_CONSULTA = r".\DatosAnypsa_Consulta.xlsx"

df_original = None
df_resultado = None

def leer_y_procesar(text_resumen):
    global df_original, df_resultado
    try:
        df_original = pd.read_excel(RUTA_EXCEL)
    except Exception as e:
        messagebox.showerror("Error", f"No se pudo leer el archivo:\n{e}")
        return

    if df_original.shape[1] < 12:
        messagebox.showerror("Error", "El archivo no tiene el número de columnas esperado.")
        return

    cols = list(df_original.columns)
    col_linea = cols[1]
    col_producto = cols[2]
    col_color = cols[3]
    col_stock = cols[6]
    col_stock_min = cols[7]
    col_precio = cols[9]
    col_demanda = cols[10]
    col_lead = cols[11]

    df_resultado = df_original.copy()
    df_resultado["Valor_Stock"] = df_resultado[col_stock] * df_resultado[col_precio]
    df_resultado["Diferencia_Stock_Minimo"] = df_resultado[col_stock] - df_resultado[col_stock_min]
    df_resultado["Cobertura_Dias"] = (df_resultado[col_stock] / (df_resultado[col_demanda] / 30)).round(1)

    try:
        df_resultado.to_excel(RUTA_SALIDA, index=False)
    except Exception as e:
        messagebox.showerror("Error", f"No se pudo guardar el archivo de salida:\n{e}")
        return

    lineas_unicas = sorted(df_resultado[col_linea].unique().tolist())
    resumen = []
    resumen.append(f"Filas: {df_resultado.shape[0]}")
    resumen.append(f"Columnas: {df_resultado.shape[1]}")
    resumen.append(f"Líneas de producto: {', '.join(lineas_unicas)}")
    resumen.append(f"Stock total: {df_resultado[col_stock].sum():.0f}")
    resumen.append(f"Valor total de inventario: {df_resultado['Valor_Stock'].sum():.2f}")
    criticos = df_resultado[df_resultado[col_stock] < df_resultado[col_stock_min]]
    resumen.append(f"Productos bajo el mínimo: {criticos.shape[0]}")

    text_resumen.config(state="normal")
    text_resumen.delete("1.0", tk.END)
    text_resumen.insert(tk.END, "\n".join(resumen))
    text_resumen.config(state="disabled")
    messagebox.showinfo("Listo", "Datos procesados y guardados en el Excel de resultados.")

def verificar_datos():
    if df_resultado is None:
        messagebox.showwarning("Atención", "Primero procesa los datos en la pestaña 1.")
        return False
    return True

def grafico_stock_por_linea():
    if not verificar_datos():
        return
    cols = list(df_resultado.columns)
    col_linea = cols[1]
    col_stock = cols[6]
    datos = df_resultado.groupby(col_linea)[col_stock].sum().sort_values(ascending=False)
    plt.figure(figsize=(8, 5))
    plt.bar(datos.index, datos.values)
    plt.xticks(rotation=45, ha="right")
    plt.title("Stock total por línea")
    plt.ylabel("Unidades")
    plt.tight_layout()
    plt.show()

def grafico_pastel_stock_linea():
    if not verificar_datos():
        return
    cols = list(df_resultado.columns)
    col_linea = cols[1]
    col_stock = cols[6]
    datos = df_resultado.groupby(col_linea)[col_stock].sum()
    plt.figure(figsize=(6, 6))
    plt.pie(datos.values, labels=datos.index, autopct="%1.1f%%")
    plt.title("Participación del stock por línea")
    plt.tight_layout()
    plt.show()

def grafico_top_productos():
    if not verificar_datos():
        return
    cols = list(df_resultado.columns)
    col_producto = cols[2]
    top = df_resultado.sort_values("Valor_Stock", ascending=False).head(10)
    plt.figure(figsize=(9, 5))
    plt.barh(top[col_producto], top["Valor_Stock"])
    plt.gca().invert_yaxis()
    plt.title("Top 10 productos por valor de inventario")
    plt.xlabel("Valor de inventario")
    plt.tight_layout()
    plt.show()

def grafico_precio_promedio_por_linea():
    if not verificar_datos():
        return
    cols = list(df_resultado.columns)
    col_linea = cols[1]
    col_precio = cols[9]
    datos = df_resultado.groupby(col_linea)[col_precio].mean().sort_values(ascending=False)
    plt.figure(figsize=(8, 5))
    plt.bar(datos.index, datos.values)
    plt.xticks(rotation=45, ha="right")
    plt.title("Precio promedio por línea")
    plt.ylabel("Precio promedio")
    plt.tight_layout()
    plt.show()

def grafico_hist_diferencia_minimo():
    if not verificar_datos():
        return
    plt.figure(figsize=(8, 5))
    plt.hist(df_resultado["Diferencia_Stock_Minimo"], bins=20)
    plt.title("Histograma: diferencia entre stock y mínimo")
    plt.xlabel("Stock - Stock mínimo")
    plt.ylabel("Frecuencia")
    plt.tight_layout()
    plt.show()

def poblar_productos_por_linea(combo_lineas, tree_linea):
    if not verificar_datos():
        return
    cols = list(df_resultado.columns)
    col_linea = cols[1]
    col_producto = cols[2]
    col_stock = cols[6]
    linea = combo_lineas.get()
    if not linea:
        messagebox.showwarning("Atención", "Selecciona una línea.")
        return
    filtrado = df_resultado[df_resultado[col_linea] == linea]
    for fila in tree_linea.get_children():
        tree_linea.delete(fila)
    for _, r in filtrado.iterrows():
        tree_linea.insert("", tk.END, values=(r[col_producto], r[col_stock]))

def consulta_avanzada(entry_color, entry_smin, entry_smax, text_resultado):
    if not verificar_datos():
        return
    cols = list(df_resultado.columns)
    col_color = cols[3]
    col_stock = cols[6]
    color = entry_color.get().strip()
    try:
        smin = int(entry_smin.get()) if entry_smin.get().strip() else None
        smax = int(entry_smax.get()) if entry_smax.get().strip() else None
    except ValueError:
        messagebox.showerror("Error", "Los valores de stock mínimo y máximo deben ser numéricos.")
        return
    df_filtro = df_resultado.copy()
    if color:
        df_filtro = df_filtro[df_filtro[col_color].str.contains(color, case=False, na=False)]
    if smin is not None:
        df_filtro = df_filtro[df_filtro[col_stock] >= smin]
    if smax is not None:
        df_filtro = df_filtro[df_filtro[col_stock] <= smax]
    try:
        df_filtro.to_excel(RUTA_CONSULTA, index=False)
    except Exception as e:
        messagebox.showerror("Error", f"No se pudo guardar el Excel de consulta:\n{e}")
        return
    text_resultado.config(state="normal")
    text_resultado.delete("1.0", tk.END)
    text_resultado.insert(tk.END, f"Filas encontradas: {df_filtro.shape[0]}\n")
    preview = df_filtro.head(20)
    text_resultado.insert(tk.END, preview.to_string(index=False))
    text_resultado.config(state="disabled")
    messagebox.showinfo("Consulta", "Consulta ejecutada. Resultado guardado en el Excel de consulta.")

def crear_interfaz():
    root = tk.Tk()
    root.title("Sistema de Análisis de Inventario ANYPSA")
    root.geometry("1050x650")

    notebook = ttk.Notebook(root)
    notebook.pack(fill="both", expand=True)

    tab1 = ttk.Frame(notebook)
    tab2 = ttk.Frame(notebook)
    tab3 = ttk.Frame(notebook)
    tab4 = ttk.Frame(notebook)
    tab5 = ttk.Frame(notebook)

    notebook.add(tab1, text="1. Procesar datos")
    notebook.add(tab2, text="2. Productos por línea")
    notebook.add(tab3, text="3. Gráficos generales")
    notebook.add(tab4, text="4. Top y precios")
    notebook.add(tab5, text="5. Consulta avanzada")

    lbl_ruta = tk.Label(tab1, text=f"Archivo de entrada:\n{RUTA_EXCEL}")
    lbl_ruta.pack(pady=10)
    btn_procesar = tk.Button(tab1, text="Leer y procesar datos", width=25,
                             command=lambda: leer_y_procesar(text_resumen))
    btn_procesar.pack(pady=10)
    text_resumen = tk.Text(tab1, height=15, width=80, state="disabled")
    text_resumen.pack(pady=10)

    frame_tab2_top = ttk.Frame(tab2)
    frame_tab2_top.pack(fill="x", pady=5)
    lbl_linea = tk.Label(frame_tab2_top, text="Línea:")
    lbl_linea.grid(row=0, column=0, padx=5, pady=5, sticky="w")
    combo_lineas = ttk.Combobox(frame_tab2_top, state="readonly", width=30)
    combo_lineas.grid(row=0, column=1, padx=5, pady=5, sticky="w")
    btn_cargar_lineas = tk.Button(frame_tab2_top, text="Cargar líneas",
                                  command=lambda: actualizar_lineas(combo_lineas))
    btn_cargar_lineas.grid(row=0, column=2, padx=5, pady=5)
    btn_ver_productos = tk.Button(frame_tab2_top, text="Ver productos",
                                  command=lambda: poblar_productos_por_linea(combo_lineas, tree_linea))
    btn_ver_productos.grid(row=0, column=3, padx=5, pady=5)

    frame_tab2_mid = ttk.Frame(tab2)
    frame_tab2_mid.pack(fill="both", expand=True, pady=10)
    cols_tree = ("Producto", "Stock")
    tree_linea = ttk.Treeview(frame_tab2_mid, columns=cols_tree, show="headings", height=18)
    tree_linea.heading("Producto", text="Producto")
    tree_linea.heading("Stock", text="Stock")
    tree_linea.column("Producto", width=450)
    tree_linea.column("Stock", width=80, anchor="center")
    tree_linea.pack(side="left", fill="both", expand=True)
    scroll_y = ttk.Scrollbar(frame_tab2_mid, orient="vertical", command=tree_linea.yview)
    tree_linea.configure(yscrollcommand=scroll_y.set)
    scroll_y.pack(side="right", fill="y")

    frame_tab2_bottom = ttk.Frame(tab2)
    frame_tab2_bottom.pack(fill="x", pady=5)
    btn_graf_linea = tk.Button(frame_tab2_bottom, text="Gráfico barras: stock por línea",
                               command=grafico_stock_por_linea)
    btn_graf_linea.pack(side="left", padx=5, pady=5)

    frame_tab3 = ttk.Frame(tab3)
    frame_tab3.pack(fill="both", expand=True, pady=10)
    btn_pastel = tk.Button(frame_tab3, text="Gráfico circular: stock por línea",
                           width=35, command=grafico_pastel_stock_linea)
    btn_pastel.pack(pady=10)
    btn_hist = tk.Button(frame_tab3, text="Histograma: diferencia stock - mínimo",
                         width=35, command=grafico_hist_diferencia_minimo)
    btn_hist.pack(pady=10)

    frame_tab4 = ttk.Frame(tab4)
    frame_tab4.pack(fill="both", expand=True, pady=10)
    btn_top = tk.Button(frame_tab4, text="Gráfico barras: top 10 productos",
                        width=35, command=grafico_top_productos)
    btn_top.pack(pady=10)
    btn_precio = tk.Button(frame_tab4, text="Gráfico barras: precio promedio por línea",
                           width=35, command=grafico_precio_promedio_por_linea)
    btn_precio.pack(pady=10)

    frame_tab5 = ttk.Frame(tab5)
    frame_tab5.pack(fill="both", expand=True, pady=10)
    lbl_color = tk.Label(frame_tab5, text="Color contiene:")
    lbl_color.grid(row=0, column=0, padx=5, pady=5, sticky="e")
    entry_color = tk.Entry(frame_tab5, width=25)
    entry_color.grid(row=0, column=1, padx=5, pady=5, sticky="w")
    lbl_smin = tk.Label(frame_tab5, text="Stock mínimo (>=):")
    lbl_smin.grid(row=1, column=0, padx=5, pady=5, sticky="e")
    entry_smin = tk.Entry(frame_tab5, width=10)
    entry_smin.grid(row=1, column=1, padx=5, pady=5, sticky="w")
    lbl_smax = tk.Label(frame_tab5, text="Stock máximo (<=):")
    lbl_smax.grid(row=2, column=0, padx=5, pady=5, sticky="e")
    entry_smax = tk.Entry(frame_tab5, width=10)
    entry_smax.grid(row=2, column=1, padx=5, pady=5, sticky="w")
    btn_consulta = tk.Button(frame_tab5, text="Ejecutar consulta",
                             command=lambda: consulta_avanzada(entry_color, entry_smin, entry_smax, text_consulta))
    btn_consulta.grid(row=3, column=0, columnspan=2, padx=5, pady=10)

    text_consulta = tk.Text(frame_tab5, height=18, width=90, state="disabled")
    text_consulta.grid(row=4, column=0, columnspan=3, padx=5, pady=5)

    root.mainloop()

def actualizar_lineas(combo_lineas):
    if not verificar_datos():
        return
    cols = list(df_resultado.columns)
    col_linea = cols[1]
    lineas_unicas = sorted(df_resultado[col_linea].unique().tolist())
    combo_lineas["values"] = lineas_unicas
    if lineas_unicas:
        combo_lineas.current(0)

if __name__ == "__main__":
    crear_interfaz()